## Импорт библиотек для анализа данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import warnings
from catboost import CatBoostRegressor, Pool, cv

## Отключаем предупреждения

In [ ]:
warnings.filterwarnings("ignore")

## Считываем данные и выводим первые 5 строк

In [ ]:
df = pd.read_csv("diamonds_train.csv")
df.head()

## Выводим основную информацию о датасете

In [55]:
print("Информация о датасете: ")
df.info()

Информация о датасете: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43018 entries, 0 to 43017
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    43018 non-null  float64
 1   cut      43018 non-null  object 
 2   color    43018 non-null  object 
 3   clarity  43018 non-null  object 
 4   depth    43018 non-null  float64
 5   table    43018 non-null  float64
 6   price    43018 non-null  int64  
 7   x        43018 non-null  float64
 8   y        43018 non-null  float64
 9   z        43018 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


## Проверем missing values

In [ ]:
print("Проверка на отсутствующие значения: ")
df.isnull().sum()

## Оцениваем статистические показатели

In [ ]:
print("Статистические показатели: ")
df.describe()

## Выделяем столбцы с признаками и столбец с целевой переменной

In [56]:
x = df.drop("price", axis=1)
y = df["price"]

## Находим категориальные признаки (необходимо для работы CatBoost)

In [57]:
cat_features = x.select_dtypes(include="object").columns.tolist()
print(f"Категориальные признаки: {cat_features}")

Категориальные признаки: ['cut', 'color', 'clarity']


## Разделяем на train и test

In [58]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
random_state=42, shuffle=True)

## Задаём параметры модели для дальнейшей их корректировки с помощью кросс-валидации

In [59]:
params = {
    "iterations": 500,
    "learning_rate": 0.1,
    "depth": 5,
    "loss_function": "RMSE",
    "eval_metric": "R2",
    "random_seed": 42,
    "verbose": 0
}

## Проводим кросс-валидацию на 5 folds

In [60]:
print("Кросс-валидация (5 фолдов): ")

train_pool = Pool(x_train, y_train, cat_features=cat_features)

cv_results = cv(
    params=params,
    pool=train_pool,
    fold_count=5,
    shuffle=True,
    seed=42,
    plot=True
)

Кросс-валидация (5 фолдов): 


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.9814417347
bestIteration = 498

Training on fold [1/5]

bestTest = 0.9803714296
bestIteration = 498

Training on fold [2/5]

bestTest = 0.9829365363
bestIteration = 496

Training on fold [3/5]

bestTest = 0.9805524092
bestIteration = 499

Training on fold [4/5]

bestTest = 0.9816221463
bestIteration = 498



## Находим, при каких параметрах модель выдавала лучшие показатели

In [61]:
best_iter_idx = cv_results["test-R2-mean"].idxmax()
best_r2 = cv_results.loc[best_iter_idx, "test-R2-mean"]
best_iteration = int(cv_results.loc[best_iter_idx, "iterations"])

## Выводим эти показатели

In [62]:
print(f"Лучшая R²: {best_r2:.4f}")
print(f"Лучшая итерация: {best_iteration}")

Лучшая R²: 0.9814
Лучшая итерация: 499


## Задаём заново параметры для модели, с учётом кросс-валидации и обучаем модель

In [63]:
final_model = CatBoostRegressor(
    iterations=best_iteration,
    learning_rate=params["learning_rate"],
    depth=params["depth"],
    loss_function=params["loss_function"],
    eval_metric=params["eval_metric"],
    random_seed=params["random_seed"],
    verbose=100,
    cat_features=cat_features
)

final_model.fit(x_train, y_train)

0:	learn: 0.1588754	total: 34.7ms	remaining: 17.3s
100:	learn: 0.9774337	total: 3.18s	remaining: 12.5s
200:	learn: 0.9806334	total: 6.05s	remaining: 8.97s
300:	learn: 0.9822126	total: 8.99s	remaining: 5.91s
400:	learn: 0.9833079	total: 12.1s	remaining: 2.96s
498:	learn: 0.9842384	total: 15.3s	remaining: 0us


## Делаем итоговое предсказание

In [64]:
y_pred = final_model.predict(x_test)

## Получаем метрики для оценки

In [65]:
test_r2 = r2_score(y_test, y_pred)
test_rmse = mean_squared_error(y_test, y_pred)

## Выводим итоговые результаты

In [66]:
print(f"Итоговые результаты: ")
print(f"R²: {test_r2:.4f}")
print(f"RMSE: {test_rmse:,.2f}")

Итоговые результаты: 
R²: 0.9807
RMSE: 311,754.01


## Сохраняем модель

In [67]:
import joblib

joblib.dump(final_model, "final_model.pkl")

['final_model.pkl']

## Считываем файл с тестовыми данными, при необходимости загружаем модель (если программа запускается заново)

In [68]:
df_test = pd.read_csv("diamonds_test.csv")

# final_model = joblib.load("final_model.pkl")

## Выводим первые 5 строк датасета

In [69]:
print("diamonds_test.csv")
df_test.head()

diamonds_test.csv


,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.02,Good,F,SI2,59.2,58.0,6.51,6.56,3.87
1,1,0.70,Very Good,I,VVS1,59.5,58.0,5.78,5.81,3.45
2,2,0.32,Very Good,H,VVS2,63.4,56.0,4.37,4.34,2.76
3,3,0.42,Ideal,F,VVS2,62.2,56.0,4.79,4.82,2.99
4,4,0.40,Ideal,F,VS2,62.3,54.0,4.74,4.77,2.96


## Выводим информацию о тестовом наборе данных

In [70]:
print("Информация о датасете: ")
df.info()

Информация о датасете: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43018 entries, 0 to 43017
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    43018 non-null  float64
 1   cut      43018 non-null  object 
 2   color    43018 non-null  object 
 3   clarity  43018 non-null  object 
 4   depth    43018 non-null  float64
 5   table    43018 non-null  float64
 6   price    43018 non-null  int64  
 7   x        43018 non-null  float64
 8   y        43018 non-null  float64
 9   z        43018 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


## Делаем проверку отсутствующих значений в датасете

In [71]:
print("Проверка на отсутствующие значения: ")
df.isnull().sum()

Проверка на отсутствующие значения: 


carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

## Выбираем нужные колонки для предсказания

In [72]:
df_test_features = df_test[x.columns]

## Делаем финальное предсказание

In [73]:
y_submission = final_model.predict(df_test_features)

submission = pd.DataFrame({
        "id": df_test["id"],
        "price": y_submission
    })

## Сохраняем финальное предсказание в файл submission.csv

In [74]:
submission.to_csv("submission.csv", index=False)
print("Сохранение submission.csv")

Сохранение submission.csv
